In [ ]:
# Calculate LRP for specific dates and plot

In [ ]:
# Important to note:!
# The use of innvestigate requires specific versions of python and keras
# The model must be built according to the versions, so the weights are saved separately
# import necesssary libaries
from pathlib import Path
import pathlib
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sns
#for plotting
import geopandas as gpd

from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator

from utils.utils_unet import *
from utils.utils_plot import plot_map
from utils.DNN_models import *
from utils.utils_LRP import *


import innvestigate
import innvestigate.utils as iutils
import innvestigate.applications.imagenet
import innvestigate.utils.visualizations as ivis

import warnings
with warnings.catch_warnings():
    warnings.simplefilter('ignore')  #catch FutureWarnings so that I can find actual errors!
    
    import keras
    import keras.backend as K
    import keras.models
    #import tensorflow as tf  # this is new for custom loss function
    from keras.models import load_model
    
    from keras.layers import Input, Dense, Activation, Dropout, Flatten, MaxPooling2D, Conv2D, Conv2DTranspose, UpSampling2D, Reshape
    from keras.models import Model
    
    from keras.utils import CustomObjectScope
    from keras.initializers import glorot_uniform

In [ ]:
import yaml
conf = yaml.safe_load(open("config.yaml"))

In [ ]:
i_shape = conf['i_shape']
o_shape = conf['o_shape']

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = conf['output_channels']
num_filters = conf['num_filters']
use_batchnorm = conf['use_batchnorm']
dropout = conf['dropout']
lr = conf['lr']

name_model = conf['model']
output_scaling = 1
output_crop = None


# load coordinates
lons_x = np.load('tmp/data/lons_y.npy')
lats_y = np.load('tmp/data/lats_y.npy')
# create a time array
#times = np.arange(np.datetime64('1979-01-01'), np.datetime64('2006-01-01')) #until validation period
#times = pd.to_datetime(times)
# load the training and testing data
#dg_train_X = np.array(xr.open_dataarray('tmp/data/dg_train_X.nc'))
#dg_train_Y = np.array(xr.open_dataarray('tmp/data/dg_train_Y.nc'))
#dg_train_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_train_Y_xtrm.nc'))

times = np.arange(np.datetime64('2016-01-01'), np.datetime64('2022-01-01')) #until validation period
times = pd.to_datetime(times)
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
#dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc'))
#dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm.nc'))

# Define hyperparameters
EPOCHS = 100
LR_METHOD = 'Constant'  # Cyclical, CosineDecay, Constant
                                            
# Default model options
opt_model = {'latent_dim': 128,
             'dropout_rate': 0.2}

# Default optimizer options
opt_optimizer = {'lr_method': 'Constant',
                 'lr': 0.0004,
                 'init_lr': 0.01}

models = {
          'UNET1': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 1, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET2': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 2, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET3': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 3, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET4': {'model': 'Unet', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 4, 'unet_use_upsample': True},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET-att': {'model': 'Unet-attention', 'run': True,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop},
                   'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET1-att': {'model': 'Unet-attention', 'run': True,
                       'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop, 'unet_depth': 1, 'unet_use_upsample': True},
                       'opt_optimizer': {'lr_method': 'Constant'}}
            }



In [ ]:
##############LRP ###########
# Select one model to evaluate the LRP
m_id = 'UNET4'
model = models[m_id]['model']
opt_model_i = models[m_id]['opt_model']
opt_optimizer_i = models[m_id]['opt_optimizer']

opt_model_new = opt_model.copy()
opt_model_new.update(opt_model_i)
opt_optimizer_new = opt_optimizer.copy()
opt_optimizer_new.update(opt_optimizer_i)


m = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True,**opt_model_new)

# compile 
m.model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer='adam', ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])

print('loads weigths')
#m.model.summary()
# load weights
m.model.load_weights('tmp/tmp_weights_DNN/UNET40.95th_trained_weights.h5')

In [ ]:
# test other depths
m_id = 'UNET1'
model = models[m_id]['model']
opt_model_i = models[m_id]['opt_model']
opt_optimizer_i = models[m_id]['opt_optimizer']
opt_model_new = opt_model.copy()
opt_model_new.update(opt_model_i)
opt_optimizer_new = opt_optimizer.copy()
opt_optimizer_new.update(opt_optimizer_i)

In [ ]:
m1 = DeepFactory_Keras(model, i_shape, o_shape, for_extremes=True,**opt_model_new)

# compile 
m1.model.compile(loss=keras.losses.categorical_crossentropy, ## instead of CategoricalCrossentropy
                  optimizer='adam', ## lr instead of learning_rate
                  metrics=['categorical_accuracy'])

print('loads weigths')
#m.model.summary()
# load weights
m1.model.load_weights('tmp/tmp_weights_DNN/UNET10.95th_trained_weights.h5')

In [ ]:
# load data
# coordinates
lons_x = np.load('tmp/data/lons_y.npy')
lats_y = np.load('tmp/data/lats_y.npy')

In [ ]:
# test-times
times = np.arange(np.datetime64('2016-01-01'), np.datetime64('2021-12-31'))
times = pd.to_datetime(times)
# load the training and testing data
dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm0.95th.nc'))

In [ ]:
#load predictions --numpy array
y_pred1_test = np.load('tmp/data/y_pred_test_UNET10.95th.npy')
y_pred4_test = np.load('tmp/data/y_pred_test_UNET40.95th.npy')
y_pred1_bool = y_pred1_test >= 0.5
y_pred1_bool = y_pred1_bool * 1
y_pred4_bool = y_pred4_test >= 0.5
y_pred4_bool = y_pred4_bool * 1

In [ ]:
#select one day # e.g. select the flood in July 2021, days: 12-15
idx = np.where(times == '2021-07-14')

In [ ]:
idx = int(idx[0])

In [ ]:
#select one day and plot
fig, ax = plt.subplots(figsize=(12, 6), ncols=2, nrows=1)
plot_map(ax[0], lons_x, lats_y, dg_test_Y_xtrm[idx,:,:,0], title="True", vmin=0, vmax=1)
plot_map(ax[1], lons_x, lats_y, y_pred4_bool[idx,:,:], title="Predicted UNet4", vmin=0, vmax=1)

In [ ]:
#fig.savefig('figures/14July2021_predicted_true.png')

In [ ]:
nx = dg_test_X.shape[1]
ny = dg_test_X.shape[2]
nchans = dg_test_X.shape[3]

In [ ]:
###Get the samples to analyze and choose a date to analyze
my_sample = dg_test_X[idx,:,:,:]
n_samples_to_analyze = 1
# Get the sample
my_sample = my_sample.reshape(n_samples_to_analyze,nx,ny,nchans)
# my  prediction for that sample
#my_estimate = y_pred_test[idx,:]

In [ ]:
#visualization
analyzer = innvestigate.create_analyzer('deep_taylor', m.model)
#analyzer = innvestigate.create_analyzer('gradient', model, neuron_selection_mode="all",allow_lambda_layers=True)
#my_analyzer = innvestigate.create_analyzer("guided_backprop", model, neuron_selection_mode = "max_activation" )

In [ ]:
#lrp_analyzerEp = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPEpsilon(model)

In [ ]:
#LRP algorithm using Alpha-Beta rule - Alpha1Beta0 only tracks positive relevance
lrp_analyzerA1B0 = innvestigate.analyzer.relevance_based.relevance_analyzer.LRPAlpha1Beta0(m.model)

In [ ]:
tes_U4= calLRP(dg_test_X[idx:idx+4,:,:,:],m.model, 'a1b0' )

In [ ]:
#tes_U1= calLRP(dg_test_X[idx:idx+4,:,:,:],m1.model, 'a1b0' )

In [ ]:
plot_xr_rel(tes_U4[0,:,:,:], lats_y, lons_x, conf['varnames'], 'example_July')